Scale Scikit-Learn for Small Data Problems
==========================================

Dask can be used scale scikit-learn to a cluster of machines for a CPU-bound problem.
We will be using a local cluster with 4 workers, each with 1 threads.

In [2]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1, memory_limit='2GB')
client

/home/joseph/.local/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41249 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:41249/status,
Dashboard: http://127.0.0.1:41249/status,Workers: 4
Total threads: 4,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36077,Workers: 4
Dashboard: http://127.0.0.1:41249/status,Total threads: 4
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:46203,Total threads: 1
Dashboard: http://127.0.0.1:36831/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:33777,


## Distributed Training


Scikit-learn uses [joblib](http://joblib.readthedocs.io/) for single-machine parallelism. This lets you train most estimators (anything that accepts an `n_jobs` parameter) using all the cores of your laptop or workstation.

Alternatively, Scikit-Learn can use Dask for parallelism.  This lets you train those estimators using all the cores of your *cluster* without significantly changing your code.  This is most useful for training large models on medium-sized datasets. 

### Create Scikit-Learn Pipeline

In [3]:
from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [4]:
# Scale Up: set categories=None to use all the categories
categories = [
    'alt.atheism',
    'talk.religion.misc',
]

print("Loading 20 newsgroups dataset for categories:")
print(categories)

data = fetch_20newsgroups(subset='train', categories=categories)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()

Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc']
857 documents
2 categories



We'll define a small pipeline that combines text feature extraction with a simple classifier.

In [5]:
pipeline = Pipeline([
    ('vect', HashingVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(max_iter=1000)),
])

In [6]:
pipeline

Pipeline(steps=[('vect', HashingVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier())])

### Define Grid for Parameter Search

Grid search over some parameters.

In [7]:
parameters = {
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    # 'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__n_iter': (10, 50, 80),
}

In [8]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=3, refit=False)

To fit this normally, we would write


```python
grid_search.fit(data.data, data.target)
```

That would use the default joblib backend (multiple processes) for parallelism.
To use the Dask distributed backend, which will use a cluster of machines to train the model, perform the fit in a `parallel_backend` context.

In [9]:
import joblib

with joblib.parallel_backend('dask'):
    grid_search.fit(data.data, data.target)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


In [10]:
grid_search.best_score_

0.9393285077495603

In [14]:
grid_search.best_params_

{'clf__alpha': 1e-06, 'tfidf__norm': 'l1', 'tfidf__use_idf': True}

In [12]:
import pandas as pd
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__alpha,param_tfidf__norm,param_tfidf__use_idf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.328223,0.014244,0.083887,0.000500,0.00001,l1,True,"{'clf__alpha': 1e-05, 'tfidf__norm': 'l1', 'tf...",0.923077,0.947552,0.940351,0.936993,0.010270,2
1,0.231422,0.023147,0.050657,0.002589,0.00001,l1,False,"{'clf__alpha': 1e-05, 'tfidf__norm': 'l1', 'tf...",0.919580,0.923077,0.891228,0.911295,0.014261,6
2,0.239779,0.007440,0.077458,0.003353,0.00001,l2,True,"{'clf__alpha': 1e-05, 'tfidf__norm': 'l2', 'tf...",0.926573,0.940559,0.922807,0.929980,0.007637,3
3,0.149948,0.013783,0.051773,0.004875,0.00001,l2,False,"{'clf__alpha': 1e-05, 'tfidf__norm': 'l2', 'tf...",0.863636,0.944056,0.908772,0.905488,0.032913,8
4,0.268456,0.005352,0.078208,0.005764,0.000001,l1,True,"{'clf__alpha': 1e-06, 'tfidf__norm': 'l1', 'tf...",0.919580,0.954545,0.943860,0.939329,0.014630,1
